# This file demonstrates using the Undatasio platform and a large language model to answer math questions from a PDF test paper.

# Installing the **Undatasio** Python API library

In [1]:
# install undatasio
!pip install -U undatasio

## To import an **UnDataIO** object, you need a token and an optional task name from the Undatasio platform.

In [2]:
from undatasio.undatasio import UnDatasIO

undatasio_obj = UnDatasIO('...', task_name='文本解析')

## The **show_version** function of the generated Undatasio object can display all version information and file lists for the current token's task name.

In [3]:
version_data = undatasio_obj.show_version()
version_data

Response(code=200, msg='success', data=      title version  count                                          file_name
0   1 files     v33      1                   [调整组合结构应对短期波折基金每日资讯20150608.pdf]
1   1 files     v32      1                   [调整组合结构应对短期波折基金每日资讯20150608.pdf]
2   1 files     v31      1                   [调整组合结构应对短期波折基金每日资讯20150608.pdf]
3   1 files     v30      1                   [调整组合结构应对短期波折基金每日资讯20150608.pdf]
4   1 files     v29      1                   [调整组合结构应对短期波折基金每日资讯20150608.pdf]
5   1 files     v28      1                   [调整组合结构应对短期波折基金每日资讯20150608.pdf]
6   1 files     v27      1          [3150f1efbaca8197be68ff6ddeb8371-已压缩.pdf]
7   1 files     v26      1                                     [adeka(1).pdf]
8   1 files     v25      1                               [初中数学浙江中考数学真题-7.pdf]
9   1 files     v24      1                                         [组合 2.pdf]
10  1 files     v23      1                               [初中数学浙江中考数学真题-7.pdf]
11  1 files     v22      

## You can use the get_result_type method of the **Undatasio** object to retrieve text information, images, tables, titles, or interline equation information from a PDF file.

In [4]:
result = undatasio_obj.get_result_type(
    type_info=['title', 'table', 'text', 'image', 'interline_equation'],
    file_name='初中数学浙江中考数学真题-7.pdf',
    version='v17'
)
print(result)

code=200 msg='success' data='https://backend.undatas.io/static/pdfParser/e8800333ea32432d86486494993378a1/v17/9f585e6a75064c27ac2a38e828168245/images//ef5fd22d77155c29b072836022db19170a3e5bc2b4c5265997ddc7b3146c0320.jpg\n\n一、选择题（本大题共10小题，每小题3分，共30分）\n1．（3分）－3的相反数是（\nA.3B.-3C.\\begin{array}{l}{{\\frac{1}{3}}}\\end{array}D.-\n2.（3分）如图，直线a，b被直线c所截，那么\\angle1的同位角是（\nhttps://backend.undatas.io/static/pdfParser/e8800333ea32432d86486494993378a1/v17/9f585e6a75064c27ac2a38e828168245/images//05d7df8aa7746c69119e040f8b67ef88eb290c9c2c7cea72278f1853d198124d.jpg\n\nA.\\angle2\\ \\mathtt{B}：3C.4D.5\n3.（3分）根据衢州市统计局发布的统计数据显示，衢州市2017年全市生产总\n值为138000000000元，按可比价格计算，比上年增长7.3\\%，数据138000000000\n元用科学记数法表示为（\nA.1.38\\!\\times\\!10^{10}元B.1.38\\!\\times\\!10^{11}元C.1.38\\!\\times\\!10^{12}元D.0.138\\!\\times\\!10^{12}元\n4.（3分）由五个大小相同的正方体组成的几何体如图所示，那么它的主视图是\nhttps://backend.undatas.io/static/pdfParser/e8800333ea32432d86486494993378a1/v17/9f585e6a75064c27ac2a38e828168245/images//c9b729f37517f9ced6ba73a90ce20470

## Install the OpenAI Python SDK, which will be used later to call the Qwen model.

In [5]:
!pip install -U openai

## Initialize the OpenAI object. You need to apply for an Alibaba Cloud API key yourself.

In [6]:
from openai import OpenAI

client = OpenAI(
    # api_key=os.getenv("DASHSCOPE_API_KEY"), 
    api_key='sk-0bfb315aa6e8424998099ca0be52cfee',
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

## Use the Bailian Qwen-max model and set the system and user prompts.

In [7]:
completion = client.chat.completions.create(
    model="qwen-max",
    messages=[
        {'role': 'system', 'content': '你是一位文本分析师，我会给你一份试卷，你需要根据用户选择的题目，将这个题目的文本信息返回。注意只返回用户提问的问题文本信息，其他的不给予返回。 试卷：%s' % (result.data, )},
        {'role': 'user', 'content': '请帮我找到“二填空题的11小题”'}],
    )
    
max_result = completion.model_dump_json()

## Use the **json** module to serialize the object returned by the Qwen-max model and extract the math problem mentioned in the user prompt.

In [8]:
import json

title_text = json.loads(max_result)['choices'][0]['message']['content']
title_text

'11．（4分）分解因式：\\(x^{2}-9\\)'

## Use the OpenAI object to query the **qwen2.5-math-72b-instruct** model with the extracted math problem and serialize the response using json to get the final answer.

In [9]:
completion = client.chat.completions.create(
    model="qwen2.5-math-72b-instruct",
    messages=[
            {'role': 'system', 'content': '你是一位数学老师，请逐步推理，并在最终答案中使用{}表示。按照markdown的格式返回文本。'},
        {'role': 'user', 'content': '请解答问题：%s' % (title_text, )}],
    )
    
math_result = completion.model_dump_json()
result = json.loads(math_result)['choices'][0]['message']['content']
print(result)

要分解因式 \(x^2 - 9\)，我们可以使用平方差公式。平方差公式表明 \(a^2 - b^2 = (a - b)(a + b)\)。

在这个例子中，我们可以确定 \(a = x\) 和 \(b = 3\)，因为 \(x^2 = (x)^2\) 和 \(9 = (3)^2\)。应用平方差公式，我们得到：
\[
x^2 - 9 = (x - 3)(x + 3)
\]

因此，\(x^2 - 9\) 的分解因式是 \(\boxed{(x-3)(x+3)}\)。
